In [ ]:
!nvidia-smi

Sun Jan 26 19:22:54 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!git clone https://github.com/huggingface/diffusers
! cd diffusers
! pip install -e .
! cd examples/text_to_image
! pip install -r requirements_sdxl.txt

Cloning into 'diffusers'...
remote: Enumerating objects: 81221, done.
remote: Counting objects: 100% (19422/19422), done.
remote: Compressing objects: 100% (1321/1321), done.
remote: Total 81221 (delta 18748), reused 18104 (delta 18099), pack-reused 61799 (from 1)
Receiving objects: 100% (81221/81221), 57.71 MiB | 34.36 MiB/s, done.
Resolving deltas: 100% (59883/59883), done.


In [ ]:
from huggingface_hub import login
login(token="***")# here we use token, but it's confidential information

In [ ]:
! pip install wandb
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### LORA FINE TUNING stable diffusion model

In [ ]:
!accelerate launch train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --dataset_name="svjack/pokemon-blip-captions-en-zh" \
  --validation_prompt="A red pokemon with ice cream." \
  --num_validation_images=7 \
  --validation_epochs=1 \
  --output_dir="/content/drive/MyDrive/output/sdxl-base-1.0-test-lora" \
  --resolution=1024 \
  --caption_column="en_text" \
  --center_crop \
  --random_flip \
  --train_text_encoder \
  --train_batch_size=1 \
  --num_train_epochs=7 \
  --max_train_steps=5000 \
  --checkpointing_steps=200 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_warmup_steps=0 \
  --report_to="wandb" \
  --dataloader_num_workers=8 \
  --allow_tf32 \
  --mixed_precision="fp16" \
  --push_to_hub \
  --hub_model_id="sdxl-base-1.0-test-lora"


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-01-26 11:08:21.496818: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-26 11:08:21.515808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-26 11:08:21.538290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cu

### ВИКОРИСТАННЯ МОДЕЛІ

In [1]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

pipe.enable_model_cpu_offload()
pipe.load_lora_weights("Alinkaaa1/sdxl-base-1.0-test-lora")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

pytorch_lora_weights.safetensors:   0%|          | 0.00/29.9M [00:00<?, ?B/s]

In [2]:
prompt = "green pokemon with flowers in the hands"
image = pipe(
    prompt = prompt,
    num_inference_steps=50,
    height=1024,
    width=1024,
    guidance_scale=7.0,
).images[0]
image.save("output_image_FT.png")


  0%|          | 0/50 [00:00<?, ?it/s]

In [3]:
# testing model without fine tuning to compare the results
pipe_1 = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

pipe_1.enable_model_cpu_offload()
image = pipe_1(
    prompt = prompt,
    num_inference_steps=50,
    height=1024,
    width=1024,
    guidance_scale=7.0,
).images[0]
image.save("output_image_ORG.png")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [7]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
import torch
from diffusers import DiffusionPipeline
def log(message):
    st.write(f"[INFO] {message}")
device = "cuda" if torch.cuda.is_available() else "cpu"
@st.cache_resource
def load_model():
    log("Завантаження моделі...")
    pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
    ).to(device)
    pipe.enable_model_cpu_offload()
    pipe.load_lora_weights("Alinkaaa1/sdxl-base-1.0-test-lora")
    log("Модель завантажено.")
    return pipe

model = load_model()
st.title('Напишіть промт-опис для генерації зображення покемона')
prompt = st.text_input("Введіть промпт:")
if st.button('Передати дані до моделі'):
    if prompt:
        st.write("Дані передано до моделі:")
        st.write(prompt)
        log("Початок генерації зображення...")
        image = model(
            prompt = prompt,
            num_inference_steps=50,
            height=1024,
            width=1024,
            guidance_scale=7.0,
        ).images[0]
        log("Генерація завершена.")
        st.image(image, caption="Зображення, згенероване LoRA", use_container_width=True)

    else:
        st.warning("Будь ласка, заповніть всі поля.")


Overwriting app.py


In [1]:
!pip install streamlit
!pip install pyngrok
!pip install transformers
!pip install torch
!pip install diffusers

In [8]:
!npm install -g localtunnel
!curl ipv4.icanhazip.com
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

⠙⠹⠸⠼
changed 22 packages in 660ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼104.154.228.186
⠙⠹⠸⠼⠴your url is: https://weak-heads-try.loca.lt
^C


### ТЕСТУВАННЯ МОДЕЛІ ТА ОБРАХУНОК МЕТРИК
(15 HOMEWORK)


Цілі:
1. Очінити візуальну якість згенерованих даних.
2. Оцінити схожість згенерованих даних на навчальні дані.
3. Оцінити відповідність згенерованих зображень текстовим запитам.

In [4]:
prompts = [
    "purple flying pokemon with wings",
    "orange fire pokemon in volcano",
    "pink water pokemon in ocean",
    "white psychic pokemon in temple",
    "brown bear-like pokemon in forest",
    "golden legendary pokemon in the sky",
    "small bug pokemon under a mushroom",
    "mysterious ghost pokemon in graveyard",
    "black dragon pokemon in mountain",
    "shiny electric pokemon in lightning storm",
    "blue fish pokemon swimming in the deep sea",
    "yellow electric pokemon on powerline",
    "giant water pokemon in a waterfall",
    "pink fairy pokemon flying in flowers",
    "green poison pokemon in jungle",
    "dark grass pokemon in the mist",
    "cute bunny pokemon hopping in field",
    "steel pokemon in factory",
    "fire-breathing dragon pokemon in cave",
    "snow pokemon in frozen forest",
    "giant insect pokemon in the jungle",
    "red fire pokemon in desert",
    "rainbow-colored pokemon flying high",
    "mysterious psychic pokemon with glowing eyes",
    "blue dragon pokemon under the moonlight",
    "legendary water pokemon in ocean",
    "green leafy pokemon in rain",
    "black and white pokemon hiding in shadows",
    "steel-armored pokemon fighting in arena",
    "giant flying pokemon above the city",
    "tiny pokemon in a flower patch",
    "snowy ice pokemon in the arctic",
    "blue ghost pokemon drifting in the air",
    "pink butterfly pokemon fluttering",
    "red fire pokemon surrounded by lava",
    "bright light pokemon shining in night sky",
    "purple psychic pokemon in desert",
    "glowing water pokemon in the night",
    "tiny bird pokemon flying over a lake",
    "purple dragon pokemon on a cliff",
    "green pokemon with glowing eyes in forest",
    "fast moving pokemon on city street",
    "cute pokemon playing in the park",
    "rainy day pokemon with umbrella",
    "strong pokemon lifting heavy rocks",
    "wild pokemon running through the grass",
    "majestic bird pokemon soaring above mountains",
    "giant rock pokemon in a cave",
    "fiery pokemon with lava background",
    "blue water pokemon in a serene pond",
    "mysterious pokemon in foggy woods",
    "legendary pokemon surrounded by aura",
    "black pokemon in a moonlit night"
]


In [5]:
import os
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

pipe.enable_model_cpu_offload()
pipe.load_lora_weights("Alinkaaa1/sdxl-base-1.0-test-lora")


output_folder = "generated_images"
os.makedirs(output_folder, exist_ok=True)

for idx, prompt in enumerate(prompts):
    image = pipe(prompt, num_inference_steps=50, height=1024, width=1024, guidance_scale=7.0).images[0]
    image_path = os.path.join(output_folder, f"generated_image_{idx}.png")
    image.save(image_path)
    print(f"Збережено зображення: {image_path}")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Збережено зображення: generated_images/generated_image_0.png


  0%|          | 0/50 [00:00<?, ?it/s]

Збережено зображення: generated_images/generated_image_1.png


...

Збережено зображення: generated_images/generated_image_50.png


  0%|          | 0/50 [00:00<?, ?it/s]

Збережено зображення: generated_images/generated_image_51.png


  0%|          | 0/50 [00:00<?, ?it/s]

Збережено зображення: generated_images/generated_image_52.png


### Perceptual Similarity (LPIPS)

In [6]:
import pandas as pd
df = pd.read_parquet("hf://datasets/svjack/pokemon-blip-captions-en-zh/data/train-00000-of-00001-78e564002aa9c8f0.parquet")

In [39]:
import os
import pandas as pd
from PIL import Image
from io import BytesIO

output_folder = "real_images"
os.makedirs(output_folder, exist_ok=True)
def save_image(image_bytes, image_id):
    image = Image.open(BytesIO(image_bytes))
    image_path = os.path.join(output_folder, f"{image_id}.png")
    image.save(image_path)

for idx, row in df.sample(50).iterrows():
    image_bytes = row['image']['bytes']
    save_image(image_bytes, idx)
    print(f"Збережено зображення: {idx}")

Збережено зображення: 515
Збережено зображення: 1
Збережено зображення: 365
Збережено зображення: 229
Збережено зображення: 60
Збережено зображення: 806
Збережено зображення: 393
Збережено зображення: 135
Збережено зображення: 252
Збережено зображення: 682
Збережено зображення: 141
Збережено зображення: 749
Збережено зображення: 382
Збережено зображення: 253
Збережено зображення: 144
Збережено зображення: 438
Збережено зображення: 241
Збережено зображення: 701
Збережено зображення: 198
Збережено зображення: 737
Збережено зображення: 344
Збережено зображення: 576
Збережено зображення: 816
Збережено зображення: 774
Збережено зображення: 367
Збережено зображення: 231
Збережено зображення: 549
Збережено зображення: 460
Збережено зображення: 703
Збережено зображення: 342
Збережено зображення: 98
Збережено зображення: 783
Збережено зображення: 126
Збережено зображення: 594
Збережено зображення: 695
Збережено зображення: 52
Збережено зображення: 738
Збережено зображення: 577
Збережено зображе

In [7]:
!pip install transformers
!pip install torch
!pip install clip-by-openai
! pip install pytorch_fid

INFO: pip is looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of clip-by-openai to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install clip-by-openai==0.1.0, clip-by-openai==0.1.1, clip-by-openai==0.1.1.2, clip-by-openai==0.1.1.3, clip-by-openai==0.1.1.4, clip-by-openai==0.1.1.5, clip-by-openai==1.0.1 and clip-by-openai==1.1 because these package versions have conflicting dependencies.

The conflict is caused by:
    clip-by-openai 1.1 depends on torch<1.7.2 and >=1.7.1
    clip-by-openai 1.0.1 depends on torch<1.7.2 and >=1.7.1
    clip-by-openai 0.1.1.5 depends on torch==1.7.1
    clip-by-openai 0.1.1.4 depends on torch==1.7.1
    clip-by-openai 0.1.1.3 depends on torch==1.7.1
    clip-by-openai 0.1.1.2 depends on torch==1.7.1
    clip-by-openai 0.1.1 depends on torch==1.7.1
    clip-by-opena

### CLIP

In [7]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

generated_images_path = "generated_images"
generated_images = np.sort([os.path.join(generated_images_path, img) for img in os.listdir(generated_images_path)])
sorted_images = sorted(generated_images, key=lambda x: int(x.split('_')[-1].split('.')[0]))
def get_features(image_path, prompt):
    image = Image.open(image_path)
    inputs = processor(text=prompt, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    image_features = outputs.image_embeds
    text_features = outputs.text_embeds
    return image_features, text_features
similarities = []
for idx, image_path in enumerate(sorted_images):
    prompt = prompts[idx]
    image_features, text_features = get_features(image_path, prompt)
    similarity_score = cosine_similarity(image_features.detach().numpy(), text_features.detach().numpy())
    similarities.append(similarity_score[0][0])
    print(f"Схожість між зображенням {image_path} і промптом: {similarity_score[0][0]:.4f}")
avg_similarity = np.mean(similarities)
print(f"Середня схожість: {avg_similarity:.4f}")

Схожість між зображенням generated_images/generated_image_0.png і промптом: 0.3458
Схожість між зображенням generated_images/generated_image_1.png і промптом: 0.3279
Схожість між зображенням generated_images/generated_image_2.png і промптом: 0.2995
Схожість між зображенням generated_images/generated_image_3.png і промптом: 0.2833
Схожість між зображенням generated_images/generated_image_4.png і промптом: 0.3229
Схожість між зображенням generated_images/generated_image_5.png і промптом: 0.2835
Схожість між зображенням generated_images/generated_image_6.png і промптом: 0.3309
Схожість між зображенням generated_images/generated_image_7.png і промптом: 0.2827
Схожість між зображенням generated_images/generated_image_8.png і промптом: 0.3358
Схожість між зображенням generated_images/generated_image_9.png і промптом: 0.3198
Схожість між зображенням generated_images/generated_image_10.png і промптом: 0.2926
Схожість між зображенням generated_images/generated_image_11.png і промптом: 0.3047
Сх

### INCEPTION SCORE

In [8]:
import torch
from torchvision import transforms
from torchvision.models import inception_v3
from torch.nn.functional import softmax
from PIL import Image
import os
import numpy as np

# Загрузка моделі InceptionV3
inception_model = inception_v3(pretrained=True).to('cuda')
inception_model.eval()

# Передобробка зображення для InceptionV3
def process_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((299, 299)),  # Розмір зображення для InceptionV3
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормалізація
    ])
    return transform(image).unsqueeze(0).to('cuda')

def calculate_inception_score(images, splits=10):
    scores = []
    for i in range(splits):
        part = images[i * len(images) // splits: (i + 1) * len(images) // splits]
        with torch.no_grad():
            logits = []
            for image in part:
                image = process_image(image)
                output = inception_model(image)
                logits.append(output)
            logits = torch.cat(logits, dim=0)
            probs = softmax(logits, dim=1)
            p_y = probs.mean(dim=0)
            kl_divergence = (probs * (probs.log() - p_y.log())).sum(dim=1)
            score = torch.exp(kl_divergence.mean())
            scores.append(score.item())
    return np.mean(scores), np.std(scores)

generated_images_path = "processed_generated_images"
generated_images = [os.path.join(generated_images_path, img) for img in os.listdir(generated_images_path)]

inception_mean, inception_std = calculate_inception_score(generated_images)
print(f"Inception Score: Mean = {inception_mean:.4f}, Std = {inception_std:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception Score: Mean = 1.5633, Std = 0.1959


FID

In [9]:
import torch
from torchvision import transforms
from pytorch_fid import fid_score
from PIL import Image
import os

def process_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

processed_generated_path = "processed_generated_images"
processed_real_path = "processed_real_images"
os.makedirs(processed_generated_path, exist_ok=True)
os.makedirs(processed_real_path, exist_ok=True)

generated_images_path = "generated_images"
real_images_path = "real_images"

generated_images = [os.path.join(generated_images_path, img) for img in os.listdir(generated_images_path)]
real_images = [os.path.join(real_images_path, img) for img in os.listdir(real_images_path)]

for idx, img_path in enumerate(generated_images):
    image = process_image(img_path).squeeze(0)
    image_pil = transforms.ToPILImage()(image)
    save_path = os.path.join(processed_generated_path, f"gen_{idx}.png")
    image_pil.save(save_path)

for idx, img_path in enumerate(real_images):
    image = process_image(img_path).squeeze(0)
    image_pil = transforms.ToPILImage()(image)
    save_path = os.path.join(processed_real_path, f"real_{idx}.png")
    image_pil.save(save_path)

fid_value = fid_score.calculate_fid_given_paths([processed_real_path, processed_generated_path], batch_size=50, device='cuda', dims=2048)
print(f"FID: {fid_value}")

100%|██████████| 2/2 [00:00<00:00,  2.69it/s]


FID: 195.00233143716733


In [10]:
import shutil

source_folder = '/content/generated_images'

destination_folder = '/content/drive/MyDrive/generated_images'

# Переміщення папки
shutil.move(source_folder, destination_folder)


'/content/drive/MyDrive/generated_images'